In [1]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

In [2]:
import findspark

In [3]:
findspark.init('/home/ubuntu/spark-2.4.3-bin-hadoop2.7/')

In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
# Start a spark session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [7]:
df = spark.read.json('people.json')

In [8]:
df.show() ## Spark automatically replaces missing values with Null

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [9]:
df.printSchema() # age and name columns

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [10]:
df.columns ## returns col names

['age', 'name']

In [11]:
df.describe().show() ## to show the statistical summary of numeric columns

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [12]:
from pyspark.sql.types import (StructField,StringType,
                               IntegerType,StructType)

In [13]:
data_schema = [StructField('age',IntegerType(), True),
               StructField('name', StringType(), True)]
## We pass in name, into this field that just means hey the column this relates to is called age
## Then you pass in some sort of class instance in this case IntegerType, meaning hey I want age to be integer type
## So we're changing from long to the integer in here.
## Third parameter is boolean, whether or not the field can be NULL. Hey it is okay that it is null.


In [14]:
## We set our final structure
final_struc = StructType(fields=data_schema)

In [15]:
# Now I am actually going to read in that json file, but I am gonna clarify that I want the schema to be my final structure
df = spark.read.json('people.json',schema=final_struc)

In [16]:
df.printSchema() ## So if you ever have issues with your own particular file formats where everything is being read 
# into as a string , you can use this methodology to actually clarify your particular schema type. 
## We will never have to do this for the course . All the course materials are in CSVs and 
## Spark has no problem inferring the schema types, but I do want you to be aware that this exists for you as a tool
## You now have a clear idea what you need to do in case you ever need to define your own schema


root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [17]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [18]:
type(df['age']) # column object

pyspark.sql.column.Column

In [19]:
type(df.select('age')) # returns data frame

pyspark.sql.dataframe.DataFrame

In [20]:
df.select('age').show() # see the singular column

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [22]:
df.head(2)[0] # type is row

Row(age=None, name='Michael')

In [24]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [27]:
# creating a new column.. A copy of age
df.withColumn('double_age',df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [29]:
df.withColumnRenamed('age','my_new_age').show() # to rename the column

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [30]:
# using sql to directly deal and interact with the data frame

In [31]:
df.createOrReplaceTempView('people') # people is registered temp view

In [32]:
results = spark.sql("SELECT * FROM people")

In [33]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [34]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")

In [35]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

